In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd


In [ ]:
# Define the path to the CSV file
csv_filename = '/content/drive/Shared drives/Capstone/tech/tech_csv_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_filename)

In [ ]:
pip install transformers[torch] -U


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers[torch]


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install accelerate -U


Note: you may need to restart the kernel to use updated packages.


# Data Extraction

In [ ]:
import json  # Add this line to import the json module
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re


In [ ]:
# Specify the path to the uploaded CSV file
csv_file_path = 'tech_csv_data.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Cleaning the dirty text!

In [ ]:
df['content'] = df['content'].astype(str)

In [ ]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove sentences containing "Click Here to find out more"
    sentences = text.split('.')
    cleaned_sentences = [sentence.strip() for sentence in sentences if 'Click Here to find out more' not in sentence]
    cleaned_text = '. '.join(cleaned_sentences)

    # Remove words with more than 30 characters
    text = ' '.join(word for word in text.split() if len(word) <= 20)

    return text


# Apply the cleaning function to the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)


In [ ]:
def remove_click_here_sentence(text):
    # Define a regular expression pattern to match sentences
    sentence_pattern = r'([^.!?]+[.!?])'

    # Find all sentences in the text
    sentences = re.findall(sentence_pattern, text)

     # Define phrases to be removed, ignoring case
    phrases_to_remove = ['click here to find out', 'all rights reserved', 'using cookies', 'menuopen', 'powered by its own proprietary technology, mashable is the go-to source for tech, digital culture and', 'reproduction without explicit permission is prohibited',
                         'addresssubscribe', '©', ' gizmodo media groupkinja is in read-only mode', 'we are working to restore service', 'need help?', 'send us a tip!', 'policyterms of servicepowered by wordpress']

    # Remove sentences containing the phrases
    filtered_sentences = [sentence.strip() for sentence in sentences
                          if all(phrase.lower() not in sentence.lower() for phrase in phrases_to_remove)]


    # Join the remaining sentences
    cleaned_text = ' '.join(filtered_sentences)

    return cleaned_text

df['cleaned_content'] = df['cleaned_content'].apply(remove_click_here_sentence)


# Custom Sentiment NLP Model (for kabeled data)

In [ ]:
pip install flair


In [ ]:
import pandas as pd
from flair.models import TextClassifier
from flair.data import Sentence


In [ ]:
# Load the pre-trained sentiment model
sentiment_model = TextClassifier.load('en-sentiment')


In [ ]:
# Function to predict sentiment
def get_sentiment(text):
    sentence = Sentence(text)
    sentiment_model.predict(sentence)
    label = sentence.labels[0]
    return label.value

In [ ]:
#df = df.head(100)

In [ ]:
# Apply the function to the DataFrame
df['label'] = df['cleaned_content'].apply(get_sentiment)

In [ ]:
# Specify the path to the uploaded CSV file
csv_file_path = 'tech_data_sentiment_labeled.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [ ]:
df = df [['cleaned_content', 'label']]
df = df[df['label'].isin(['POSITIVE', 'NEGATIVE'])]


In [ ]:
df

,cleaned_content,label
0,"Richard Branson Announces New Space Company, T...",NEGATIVE
1,"No, Poppy Seeds Probably Aren't Going to Make ...",NEGATIVE
2,Grab an Amazon Echo for half off todayEngadget...,NEGATIVE
3,Vodafone shows again that own-brand phones can...,NEGATIVE
4,Apple’s Lisa Jackson to talk about the environ...,NEGATIVE
...,...,...
13858,Turing Award Winner Martin Hellman On Cryptogr...,POSITIVE
13859,President Barack Obama throws in a 'Game of Th...,NEGATIVE
13860,"By Jo Rhett April 25, 2016Departure board at n...",NEGATIVE
13861,By Adam Rosenberg2016-05-08 21:07:15 UTCIf vir...,POSITIVE


# Fine-tune LLM

In [ ]:
def read_imdb_split(split_dir):
  split_dir = Path(split_dir)
  texts = []
  labels = []
  for label_dir in ["pos", "ned"]:
    for text_file in (split_dir/label_dir).iterdir():
      texts.append(text_file.read_text())
      labels.append(0 if label_dir == "neg" else 1)

  return texts, labels

In [ ]:
# Convert the DataFrame to lists
texts = df['cleaned_content'].tolist()
labels = df['label'].apply(lambda x: 1 if x == 'POSITIVE' else 0).tolist()

# Display the result
print("Texts:", texts)
print("Labels:", labels)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
len(labels)

13833

In [ ]:
type(labels)

list

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and testing sets
#train_texts, test_texts, train_labels, test_labels = train_test_split(
#    texts, labels, test_size=0.2, random_state=42
#)

In [ ]:
#and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [ ]:
type(train_texts[0])

str

In [ ]:
type(val_texts)

list

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    accuracy = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
#creating PyTorch Dataset
class FlairNewsDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast. from_pretrained(model_name)

In [ ]:
#ensure that all our sequences are padded to the same length and are truncated to be no longer than model's
#maximum input length. This will allow us to feed batches of sequences into the model at the same time.

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
#test_encodings = tokenizer(test_texts, truncation=True, padding=True)


In [ ]:
train_dataset = FlairNewsDataset(train_encodings, train_labels)
val_dataset = FlairNewsDataset(val_encodings, val_labels)
#test_dataset = FlairNewsDataset(test_encodings, test_labels)


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

2024-05-28 21:00:25.359115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 21:00:29.590699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-28 21:00:29.592299: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"  # Evaluate at the end of each epoch
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
# Start training
train_output = trainer.train()

# Access training logs
train_logs = train_output.metrics
print(train_logs)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.429400,0.390543,0.815323,0.741276,0.675439,0.706827
2,0.260300,0.399343,0.827973,0.731423,0.755482,0.743258
3,0.082900,0.735297,0.833755,0.736402,0.771930,0.753747


{'train_runtime': 25858.6279, 'train_samples_per_second': 1.284, 'train_steps_per_second': 0.08, 'total_flos': 4397652700581888.0, 'train_loss': 0.2892571007097158, 'epoch': 3.0}


In [ ]:
# Evaluate the model
eval_output = trainer.evaluate()
print(eval_output)

{'eval_loss': 0.7352970242500305, 'eval_accuracy': 0.8337549692808095, 'eval_precision': 0.7364016736401674, 'eval_recall': 0.7719298245614035, 'eval_f1': 0.7537473233404711, 'eval_runtime': 666.4462, 'eval_samples_per_second': 4.152, 'eval_steps_per_second': 0.066, 'epoch': 3.0}


In [ ]:
# Save the final model and tokenizer
model.save_pretrained('/content/drive/Shared drives/Capstone/final_model')
tokenizer.save_pretrained('/content/drive/Shared drives/Capstone/final_tokenizer')

In [ ]:
# Specify the local directory to save the model and tokenizer
local_model_dir = './sa_model'
local_tokenizer_dir = './sa_tokenizer'

# Save both the model and tokenizer locally
model.save_pretrained(local_model_dir)
tokenizer.save_pretrained(local_tokenizer_dir)

('./sa_tokenizer/tokenizer_config.json',
 './sa_tokenizer/special_tokens_map.json',
 './sa_tokenizer/vocab.txt',
 './sa_tokenizer/added_tokens.json',
 './sa_tokenizer/tokenizer.json')

# Testing the fine-tuned Model

In [ ]:

# Load the saved model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained('/content/drive/Shared drives/Capstone/final_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('/content/drive/Shared drives/Capstone/final_tokenizer')

In [ ]:

# Define the input text for sentiment analysis
input_text = "This movie is fantastic! I loved every minute of it."

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate predictions
outputs = model(**inputs)

# Interpret results
predicted_class = outputs.logits.argmax().item()

# Define mapping for sentiment labels
sentiment_labels = {0: 'Negative', 1: 'Positive'}

# Print the predicted sentiment
print("Predicted Sentiment:", sentiment_labels[predicted_class])

In [ ]:

# Ensure the model is in evaluation mode
model.eval()

# Define the mapping for sentiment labels
sentiment_labels = {0: 'Negative', 1: 'Positive'}

In [ ]:
# Define the prediction function
def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the predicted class
    predicted_class = outputs.logits.argmax().item()
    # Return the predicted sentiment
    return sentiment_labels[predicted_class]

In [ ]:
df = df.head(10)

In [ ]:
# Apply the function to each row in the 'cleaned_content' column
df['predicted_sentiment'] = df['cleaned_content'].apply(predict_sentiment)


In [ ]:
df